In [2]:
%pylab


Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [31]:
import scipy.io as spio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

trimble_data = loadmat('/media/erik/3tb/RCV_data/arlanda_test_track_wireshark_18032016/Loop1.mat')

/usr/local/lib/python2.7/dist-packages/scipy/io/matlab/mio.py:135: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [46]:
print trimble_data["Results"]["GPS"]["signalNames"]

[u'GPS/Append Timestamp to GPS/GPS_Latitude'
 u'GPS/Append Timestamp to GPS/GPS_LatitudeTimestamp'
 u'GPS/Append Timestamp to GPS/GPS_Longitude'
 u'GPS/Append Timestamp to GPS/GPS_LongitudeTimestamp'
 u'GPS/Append Timestamp to GPS/GPS_Altitude'
 u'GPS/Append Timestamp to GPS/GPS_AltitudeTimestamp'
 u'GPS/Append Timestamp to GPS/GPS_Heading'
 u'GPS/Append Timestamp to GPS/GPS_HeadingTimestamp'
 u'GPS/Append Timestamp to GPS/GPS_VehicleSpeed'
 u'GPS/Append Timestamp to GPS/GPS_VehicleSpeedTimestamp'
 u'GPS/Append Timestamp to GPS/GPS_Status_Gain'
 u'GPS/Append Timestamp to GPS/GPS_Quality_Gain'
 u'GPS/Append Timestamp to GPS/GPS_Satellites'
 u'GPS/Append Timestamp to GPS/UTCtime_hhmmsscc_Gain'
 u'GPS/Append Timestamp to GPS/UTCtime_ddmmyy_Gain' u'Time']


In [3]:
from LatLongUTMconversion import *

gps_fields = {'lat':0,'lat_ts':1,'lon':2,'lon_ts':3,'heading':6}

#print trimble_data["Results"]["GPS"]["signalNames"]
data =  trimble_data["Results"]["GPS"]["data"]
print np.shape(data)
lat = data[:,gps_fields['lat']]
lat_ts = data[:,gps_fields['lat_ts']]
lon = data[:,gps_fields['lon']]
lon_ts = data[:,gps_fields['lon_ts']]
lat_ok = lat>0
lat = lat[lat_ok]
lat_ts = lat_ts[lat_ok]
lon = lon[lat_ok]
lon_ts = lon_ts[lat_ok]
heading = data[:,gps_fields['heading']]
#heading from north in degrees (clockwise...)
heading = heading[lat_ok]
heading = np.pi/180*heading
heading = np.pi/2-heading

N = len(lat)
utm = np.zeros((N,2)) #assume all the same zone
for i in range(0,N):
    (z,e,n) = LLtoUTM(23,lat[i],lon[i])
    utm[i,0] = e
    utm[i,1] = n



plot(utm[:,0],utm[:,1])
#plot(lat_ts/1.0e3-lon_ts/1.0e3)

for i in range(0,N,40):
    plot([utm[i,0],utm[i,0]+np.cos(heading[i])],
         [utm[i,1],utm[i,1]+np.sin(heading[i])],'r')



(20804, 16)


In [59]:
plot(np.diff(lat_ts))
plot(np.diff(lat_ts),'.')

In [3]:
#Load velodyne timestamps (seconds since last hour)
velodyne_secs = np.genfromtxt('sec_since_last_h_loop1.csv')

In [6]:
plot(np.diff(velodyne_secs))
plot(np.diff(velodyne_secs),'g.')

In [17]:
#This says 0.2 seconds between scans.. which is wrong...
#What is the time between packets in wireshark??? (79 packets/scan)

#The time stamp represents the time of the first shot of 
#the first firing sequence

#WS time 0.002664
ts_chars1 = ['4e','a1','1c','8e']
ts_us1 = int(''.join(ts_chars1[::-1]),16)

#WS time 0.003316
ts_chars2 = ['e5','a3','1c','8e']
ts_us2 = int(''.join(ts_chars2[::-1]),16)

print "ts_us1 = ",ts_us1
print "ts_us2 = ",ts_us2
print "diff_us = ",ts_us2-ts_us1

laser_cycle_us = 55.296
data_blocks=12
packet_cycle_us = data_blocks*laser_cycle_us

print "packet_cycle_us=",packet_cycle_us

packets_per_scan = 79

print "scan_cycle_us=",packets_per_scan*packet_cycle_us
print "scan_cycle_s=",packets_per_scan*packet_cycle_us/1.0e6

print 1/0.052420608
print 1/0.052420608/2

ts_us1 =  2384240974
ts_us2 =  2384241637
diff_us =  663
packet_cycle_us= 663.552
scan_cycle_us= 52420.608
scan_cycle_s= 0.052420608
19.0764670261
9.53823351305


In [27]:
#WS time 0.057735
ts_chars1 = ['70','78','1d','8e']
ts_us1 = int(''.join(ts_chars1[::-1]),16)

#WS time 0.191120
ts_chars2 = ['08','7b','1d','8e']
ts_us2 = int(''.join(ts_chars2[::-1]),16)

ts_chars3 = ['06','84','1f','8e']
ts_us3 = int(''.join(ts_chars3[::-1]),16)

ts_chars4 = ['9d','86','1f','8e']
ts_us4 = int(''.join(ts_chars4[::-1]),16)

print "ts_us1 = ",ts_us1
print "ts_us2 = ",ts_us2
print "ts_us3 = ",ts_us3
print "ts_us4 = ",ts_us4
print "diff_us = ",ts_us2-ts_us1
print "diff_us = ",ts_us3-ts_us2
print "diff_us = ",ts_us4-ts_us3

ts_us1 =  2384296048
ts_us2 =  2384296712
ts_us3 =  2384430086
ts_us4 =  2384430749
diff_us =  664
diff_us =  133374
diff_us =  663


In [30]:
#microseconds from velodyne packets
velodyne_us = np.genfromtxt('velodyne_us.csv')

#plot(np.diff(velodyne_us))
#plot(np.diff(velodyne_us),'.')

plot(velodyne_us)
plot(velodyne_us,'.')

In [29]:
velodyne_us2 = np.genfromtxt('velodyne_us2.csv')

plot(np.diff(velodyne_us2))
plot(np.diff(velodyne_us2),'.')

In [32]:
import datetime
import time

#this is time in ms since 2004-01-01
ts0 = lat_ts[0]

s = "01/01/2004"
t0 = time.mktime(datetime.datetime.strptime(s,"%d/%m/%Y").timetuple())

print "t0 = ",t0
print "ts0/1e3 = ",ts0/1e3

t_unix = t0 + ts0/1e3

curr_date = datetime.datetime.fromtimestamp(t_unix).strftime("%d/%m/%Y %H:%M:%S.%f")
print "curr_date = ",curr_date


t0 =  1072911600.0
ts0/1e3 =  385418412.45
curr_date =  18/03/2016 20:40:12.450000


In [30]:
t_unix = t0 + lat_ts/1e3
plot(np.diff(t_unix))
plot(np.diff(t_unix),'g.')

In [33]:
#Let's plot the seconds since last hour for the trimble data
#and for the velodyne data
curr_date_hour = datetime.datetime.fromtimestamp(t_unix).strftime("%d/%m/%Y %H")
print curr_date_hour
start_hour_unixtime = time.mktime(datetime.datetime.strptime(curr_date_hour,"%d/%m/%Y %H").timetuple())

print t_unix
print start_hour_unixtime

sec_since_last_h_trimble  = t0 + lat_ts/1e3 - start_hour_unixtime


t_ticks = np.arange(np.min([velodyne_secs[0],sec_since_last_h_trimble[0]]),sec_since_last_h_trimble[-1]+1.0,0.001)
num_ticks = np.size(t_ticks)

trimble_trig = np.zeros(num_ticks)
trimble_trig[np.searchsorted(t_ticks,sec_since_last_h_trimble)]=1.0

velodyne_trig = np.zeros(num_ticks)
velodyne_trig[np.searchsorted(t_ticks,velodyne_secs)]=1.0

plot(t_ticks[trimble_trig==1.0],1.01*trimble_trig[trimble_trig==1.0],'r*')
plot(t_ticks[velodyne_trig==1.0],0.99*velodyne_trig[velodyne_trig==1.0],'bo')

ylim(0.7,1.3)

#plot(sec_since_last_h_trimble,'r')
#plot(velodyne_secs,'b')
#print velodyne_secs



18/03/2016 20
1458330012.45
1458327600.0


(0.7, 1.3)

In [57]:
trimble_data1 = loadmat('/media/erik/3tb/RCV_data/arlanda_test_track_wireshark_18032016/Loop1.mat')
trimble_data2 = loadmat('/media/erik/3tb/RCV_data/arlanda_test_track_wireshark_18032016/Loop2.mat')
#trimble_data2 = loadmat('/media/erik/3tb/RCV_data/arlanda_test_track_wireshark_18032016/ThirdRunBehindTruckStephanos30and50.mat')

In [58]:
gps_fields = {'lat':0,'lat_ts':1,'lon':2,'lon_ts':3,'heading':6}
data1 =  trimble_data1["Results"]["GPS"]["data"]
lat_ts1 = data1[:,gps_fields['lat_ts']]

data2 =  trimble_data2["Results"]["GPS"]["data"]
lat_ts2 = data2[:,gps_fields['lat_ts']]

print "lat_ts1[0] = ",lat_ts1[0]
print "lat_ts2[0] = ",lat_ts2[0]

lat_ts1[0] =  385418715764.0
lat_ts2[0] =  385418715764.0


In [59]:
from LatLongUTMconversion import *


def plot_trimble(data,pc='b'):
    lat = data[:,gps_fields['lat']]
    lat_ts = data[:,gps_fields['lat_ts']]
    lon = data[:,gps_fields['lon']]
    lon_ts = data[:,gps_fields['lon_ts']]
    lat_ok = lat>0
    lat = lat[lat_ok]
    lat_ts = lat_ts[lat_ok]
    lon = lon[lat_ok]
    lon_ts = lon_ts[lat_ok]
    heading = data[:,gps_fields['heading']]
    #heading from north in degrees (clockwise...)
    heading = heading[lat_ok]
    heading = np.pi/180*heading
    heading = np.pi/2-heading

    N = len(lat)
    utm = np.zeros((N,2)) #assume all the same zone
    for i in range(0,N):
        (z,e,n) = LLtoUTM(23,lat[i],lon[i])
        utm[i,0] = e
        utm[i,1] = n

    plot(utm[:,0],utm[:,1],pc)

plot_trimble(data1)
plot_trimble(data2,'r')